In [ ]:
import struct
import time
import csv
from scapy.all import *
from datetime import datetime

# Data Reading from pcap file
try:
    packets = rdpcap('scanner_valid_testdata.pcap')
except FileNotFoundError:
    print("Error: File not found.")
    exit()

# Setup CSV Logging
csv_file = open('scanner_log_output.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Timestamp', 'Motor Speed (RPM)', 'Return Mode'])

# A map to make the return mode human-readable
return_mode_map = {
    0x39: "Dual Return (Last, Strongest)",
    0x37: "Single Return (Strongest)",
}


print(" Starting Scanner Data Processing (logging to scanner_log.csv) " )

for packet in packets:
    if packet.haslayer(Raw):
        udp_payload = packet[Raw].load
        if len(udp_payload) == 820:
            # Parsing the Motor Speed
            motor_speed = struct.unpack('<H', udp_payload[803:805])[0]

            # Parsing the Return Mode
            return_mode_code = udp_payload[802]
            return_mode_text = return_mode_map.get(return_mode_code, f"Unknown ({hex(return_mode_code)})")
            
            # Parsing the Timestamp
            date_parts = struct.unpack('<6B', udp_payload[805:811])
            timestamp_us = struct.unpack('<I', udp_payload[811:815])[0]
            timestamp = datetime(
                year=date_parts[0] + 1900, month=date_parts[1], day=date_parts[2],
                hour=date_parts[3], minute=date_parts[4], second=date_parts[5],
                microsecond=timestamp_us
            )

            # Write the data to the CSV file
            csv_writer.writerow([timestamp, motor_speed, return_mode_text])

# --- Cleanup ---
csv_file.close()
print(" Data saved to scanner_log.csv")



 Starting Scanner Data Processing (logging to scanner_log.csv) 
 Data saved to scanner_log.csv
